# `make_attn_mask`（TimesFM / Flax）

`make_attn_mask`は、Transformerの注意(Attention)で使う **マスク(mask)** を作ります。

- **因果マスク(causal mask)**：未来トークン(右側)を見ないようにする  
  → `q_index >= kv_index`
- **左パディング(left padding)の無視**：系列の先頭にある無効パッチ(例：パディング)を見ない  
  → `kv_index >= num_all_masked_kv`

返り値は `bool` の配列で、形状は **`[b, 1, q, n]`** です。
- `b`：バッチサイズ
- `q`：`query_length`
- `n`：`kv_length`（ただし `kv_length=0` のときは `query_length` と同じ）

> 注意：`query_length` と `kv_length` は `jax.jit` の `static_argnames` なので **Python の `int` を渡す必要**があります（`None`やJAX配列は不可）。


## 1) 学習/通常推論（キャッシュなし・自己注意：kv_length=0 → kv_length=query_length扱い）

In [ ]:
import jax.numpy as jnp
from timesfm.flax.transformer import make_attn_mask

# 例：バッチサイズ b=2
query_length = 8  # int（必須）
num_all_masked_kv = jnp.array([0, 2], dtype=jnp.int32)  # 先頭から無視するKV数（左パディング数）

attn_mask = make_attn_mask(
    query_length=query_length,
    num_all_masked_kv=num_all_masked_kv,
    query_index_offset=None,
    kv_length=0,  # 0ならkv_length=query_lengthになる
)

print(attn_mask.shape)  # (2, 1, 8, 8)
print(attn_mask.dtype)  # bool
attn_mask

(2, 1, 8, 8)
bool


## 2) デコード/キャッシュあり（query_index_offsetとkv_lengthを使う）

In [3]:
import jax.numpy as jnp
from timesfm.flax.transformer import make_attn_mask

b = 2
query_length = 4      # 今回追加するクエリ長（例：新しいパッチ数）
kv_length = 16        # キャッシュに既にあるKV長（例：decode_cache_size）
num_all_masked_kv = jnp.array([0, 1], dtype=jnp.int32)  # 左パディング数
query_index_offset = jnp.array([8, 12], dtype=jnp.int32)  # 各バッチの“今回の書き込み開始位置”(next_index相当)

attn_mask = make_attn_mask(
    query_length=query_length,
    num_all_masked_kv=num_all_masked_kv,
    query_index_offset=query_index_offset,
    kv_length=kv_length,
)

print(attn_mask.shape)  # (2, 1, 4, 16)


(2, 1, 4, 16)


## ForecastConfig (予測設定)
予測時のパラメータ（ホライズン、コンテキスト長、正規化など）を管理するクラスです。

In [4]:
from timesfm.configs import ForecastConfig

# 予測設定の作成
# モデルのデフォルトに合わせて設定します（例: context=512, horizon=128）
forecast_config = ForecastConfig(
    max_context=512,          # モデルに入力する最大過去データ点数
    max_horizon=128,          # 一度に予測する最大ステップ数
    normalize_inputs=True,    # 入力データを正規化するか (RevIN等)
    window_size=0,            # 分解予測時のウィンドウサイズ (0は無効)
    per_core_batch_size=32,   # コアごとのバッチサイズ
    use_continuous_quantile_head=False, # 連続分位点ヘッドの使用有無
    force_flip_invariance=True, # 反転不変性を強制するか (時系列の上下反転に対して頑健にする)
    infer_is_positive=False,    # 出力が非負であることを強制するか
    fix_quantile_crossing=True, # 分位点の交差（矛盾）を修正するか
    return_backcast=False       # 過去データの再構成を返すか
)

print(f"ForecastConfig: context={forecast_config.max_context}, horizon={forecast_config.max_horizon}")

ForecastConfig: context=512, horizon=128


## StackedTransformersConfig & TransformerConfig

In [5]:
from timesfm.configs import TransformerConfig, StackedTransformersConfig

# 1. 個別のTransformer層の詳細設定
transformer_config = TransformerConfig(
    model_dims=1280,      # モデル次元 (d_model)
    hidden_dims=1280,     # FFNの隠れ次元
    num_heads=16,         # 注意ヘッド数
    attention_norm='rms', # Attention正規化方式 (RMSNorm)
    feedforward_norm='rms', # FFN正規化方式
    qk_norm='rms',        # Query/Key正規化方式
    use_bias=False,       # 線形層にバイアスを使うか
    use_rotary_position_embeddings=True, # RoPE (回転位置埋め込み)
    ff_activation='swish',# FFN活性化関数
    fuse_qkv=True         # QKVを融合実装するか (高速化)
)

# 2. Transformerを積み上げる設定
stacked_config = StackedTransformersConfig(
    num_layers=20,                 # 積み上げる層数
    transformer=transformer_config # 上記の設定を適用
)

print(f"Model Config: {stacked_config.num_layers} layers, {transformer_config.model_dims} dims")

Model Config: 20 layers, 1280 dims


## 2. Flax ユーティリティ (Flax Utilities)
時系列特有の前処理やAttentionマスク生成などの低レイヤー関数です。

revin (Reverse Instance Normalization)
時系列データの分布シフトに対処するため、入力データを平均・分散で正規化し、出力後に逆変換するための関数です。

In [6]:
import jax
import jax.numpy as jnp
from timesfm.flax.util import revin

# ダミーデータ: [batch=2, seq_len=10, dim=1]
x_dummy = jnp.array([
    [10.0, 11.0, 12.0, 13.0, 14.0],
    [100.0, 101.0, 102.0, 103.0, 104.0]
]).reshape(2, 5, 1)

# 統計量 (ここでは単純化のため手動設定、通常は計算して求める)
mu = jnp.mean(x_dummy, axis=1, keepdims=True)
sigma = jnp.std(x_dummy, axis=1, keepdims=True) + 1e-6

# 1. 正規化 (Forward)
x_norm = revin(x=x_dummy, mu=mu, sigma=sigma, reverse=False)
print("Normalized (mean approx 0):\n", x_norm[0].flatten())

# 2. 逆変換 (Reverse)
x_recon = revin(x=x_norm, mu=mu, sigma=sigma, reverse=True)
print("Reconstructed:\n", x_recon[0].flatten())

# 元に戻っているか確認
assert jnp.allclose(x_dummy, x_recon, atol=1e-5)
print("RevIN check passed.")

Normalized (mean approx 0):
 [-1.4142126 -0.7071063  0.         0.7071063  1.4142126]
Reconstructed:
 [10. 11. 12. 13. 14.]
RevIN check passed.


## make_attn_mask (Attention Mask)
TransformerのSelf-Attentionで使用する因果マスクを作成します。

In [7]:
from timesfm.flax.transformer import make_attn_mask

# バッチサイズ2, クエリ長8
query_len = 8
num_masked_kv = jnp.array([0, 2], dtype=jnp.int32) # バッチ内の各サンプルで無視する先頭トークン数

mask = make_attn_mask(
    query_length=query_len,
    num_all_masked_kv=num_masked_kv,
    kv_length=0 # 0の場合 query_length と同じとみなされる
)

print(f"Mask shape: {mask.shape}") # (2, 1, 8, 8)
# マスクの可視化 (1つ目のサンプル)
print("Mask sample 0 (True=Attend, False=Masked):\n", mask[0, 0].astype(int))

Mask shape: (2, 1, 8, 8)
Mask sample 0 (True=Attend, False=Masked):
 [[1 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0]
 [1 1 1 1 0 0 0 0]
 [1 1 1 1 1 0 0 0]
 [1 1 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1]]


## 3. TimesFM 2.5 モデル利用 (Main Model Usage)
高レベルAPIを使用したモデルのロードと推論のワークフローです。

TimesFM_2p5 クラスの初期化とチェックポイントロード
これは flax と torch のバックエンドをラップする高レベルクラスである可能性がありますが、ここでは from_pretrained メソッドを持つ TimesFM_2p5_200M_flax を例にします。

In [9]:
from timesfm.timesfm_2p5.timesfm_2p5_flax import TimesFM_2p5_200M_flax

# モデルのインスタンス化
# 注意: 実際にHugging Faceからダウンロードする場合は時間がかかります
tfm_model = TimesFM_2p5_200M_flax()

# 事前学習済みモデルのロード (例)
tfm_model.from_pretrained(
    model_id='google/timesfm-2.5-200m-flax',
    force_download=False
)
print("TimesFM model instance created.")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

99d9838f4ea666b0baf271caec0acb55:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

_CHECKPOINT_METADATA:   0%|          | 0.00/262 [00:00<?, ?B/s]

_sharding: 0.00B [00:00, ?B/s]

process_0: 0.00B [00:00, ?B/s]

...

manifest.ocdbt:   0%|          | 0.00/117 [00:00<?, ?B/s]

(…)uid-1268afaf-1c19-4568-8171-17e9c4ca2504:   0%|          | 0.00/45.0 [00:00<?, ?B/s]

manifest.ocdbt:   0%|          | 0.00/266 [00:00<?, ?B/s]

f0a794078cf86d67e5026770a0cb3aaa:   0%|          | 0.00/590 [00:00<?, ?B/s]

TimesFM model instance created.


## forecast (予測実行)
入力データを与えて未来を予測します。ここではモックデータでの呼び出しイメージを示します。

In [10]:
import numpy as np

# ダミーの時系列データ (リスト形式)
# 2つの時系列: 長さ 64
inputs = [
    np.sin(np.linspace(0, 20, 64)),
    np.cos(np.linspace(0, 20, 64)) * 10
]

# コンパイル (JITコンパイルが走るため初回は遅い)
# 注意: 実際の重みがロードされていないとランダムな出力になります
tfm_model.compile(forecast_config=forecast_config)

# 予測実行 (コメントアウトを外して実行)
forecast_result = tfm_model.forecast(
    horizon=32,
    inputs=inputs
)

# print("Forecast shape:", forecast_result.shape) # (2, 32, output_dim)

Compiling model...


/home/az/miniconda3/envs/ts/lib/python3.11/site-packages/jax/_src/interpreters/mlir.py:1274: UserWarning: Some donated buffers were not usable: float32[1,32,16,128,10], float32[1,32,16,128,10], float32[32,1], float32[32,1].
See an explanation at https://docs.jax.dev/en/latest/faq.html#buffer-donation.
  warnings.warn("Some donated buffers were not usable:"


Compiling done.


## 4. Torch コンポーネント (Torch Components)
PyTorch環境でのコンポーネント確認です。

MultiHeadAttention (Torch)
PyTorch版のMulti-Head Attentionモジュールの動作確認です。

In [11]:
import torch
from timesfm.torch.transformer import MultiHeadAttention

# 設定
dim = 64
heads = 4

# モジュール初期化
mha = MultiHeadAttention(
    num_heads=heads,
    in_features=dim,
    use_per_dim_scale=True,
    use_rotary_position_embeddings=True,
    use_bias=False,
    qk_norm='rms'
)

# ダミー入力 [Batch, Length, Dim]
x_torch = torch.randn(2, 16, dim)

# Forward
output = mha.forward(
    inputs_q=x_torch,
    decode_cache=None,
    patch_mask=None
)

print(f"Torch MHA Output shape: {output.shape}") # (2, 16, 64)

AttributeError: 'tuple' object has no attribute 'shape'

## 5. 共変量 (XReg / Covariates)
TimesFMは外部共変量（休日、天気、カテゴリ情報など）を扱うために、In-Contextでの線形回帰などを組み合わせる XReg モジュールを持っています。

In [ ]:
from timesfm.utils.xreg_lib import BatchedInContextXRegLinear

# データ準備
# ターゲット: [2つの系列, 長さ100]
targets = [
    [x * 0.1 + 2.0 for x in range(100)],
    [x * -0.05 + 10.0 for x in range(100)]
]
train_lens = [80, 80] # 学習期間
test_lens = [20, 20]  # テスト（予測）期間

# 静的共変量（各系列に紐づく固定値、例: 店舗IDの埋め込みなど）
# ここではダミー
static_numerical = [[1.0], [0.5]]

# XRegモデルの初期化
xreg = BatchedInContextXRegLinear(
    targets=targets,
    train_lens=train_lens,
    test_lens=test_lens,
    static_numerical_covariates=static_numerical,
    # 動的共変量がある場合は以下に追加
    # train_dynamic_numerical_covariates=...,
)

# フィッティング (Ridge回帰など)
xreg.fit(ridge=1.0)

print("XReg fitting completed.")
# 内部では共変量行列を作成し、最小二乗法などで係数を推定しています

## normalize (正規化ユーティリティ)

In [ ]:
from timesfm.utils.xreg_lib import normalize

batch_data = np.array([
    [10, 20, 30],
    [100, 200, 300]
], dtype=float)

# バッチごとの正規化
normalized_batch, stats = normalize(batch_data)

print("Mean:", stats.mean)
print("Std:", stats.std)
print("Normalized:\n", normalized_batch)